### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import statistics
import scipy
import statistics as st
import statsmodels.stats.proportion 

import warnings
warnings.filterwarnings("ignore")

##  <ins>Задание 1.</ins>

**Контекст**

Мы много платим оператору из-за того, что он сам по каждому заданию звонит клиенту и ждёт ответа клиента. Мы хотим попробовать снизить затраты на операторов, внедрив систему, которая будет сама звонить человеку и в случае успеха перенаправлять задание оператору.
        
**Тест**

Мы проводим тест, в рамках которого мы разбиваем весь поток на две части рановероятно, где одной части (контролю) мы продаём  без автоматической системы дозвона, а второй части (тесту) - с ней. Целью этого теста является снижение расходов на задание.

**Статистические вводные**

Уровень значимости 5%.

Мы хотели бы различать 5%-ное 
изменение целевой метрики
с вероятностью 90%.

### 1. Необходимо выбрать метрику и привести аргументацию.

**Целевая метрика** = среднее значение расходов (Σ расходов / кол-во задач)

**Контрольная метрика** = среднее значение заработка (Σ PV / кол-во задач); Количество продаж.

Для бизнеса главное - рост прибыли. Знаем, что 
NPV = PV - Расходы. Значит если начнут падат расходы, то будет увеличиваться NPV. Также по условию целью теста является снижение расходов. Поэтому среднее значение расходов - целевая матрика. 

Если упадет значение PV, то мы не сможем понять, за счет чего изменилось значение прибыли. Поэтому среднее PV - контрольная метрика. Также контрольной метрикой можно сделать количество успешно завешенных продаж.

### 2. Альтернатива в критерии.

**Гипотеза H(0)** - среднее значение расходов на тесте и контроле будет одинаковым

**Альтернатива H(1)** - среднее значение расходов для тестовой выборки будет ниже на 5% и более

### 3. Каков размер выборки? Привести аргументацию и написать как получилось то или иное число.

$$n = \frac{(Z_{1 - α} - Z_{1 - β})^2 * σ^2}{k * Δ^2}$$ 

Из условия задачи знаем:
1.   Уровень α = 0.05
2.   Уровень β = 0.9
3.   k = 0.5

Не знаем σ, но можем посчитать выборочную дисперсию.

Δ = MDE

In [2]:
hist_df = pd.read_csv('hist_telesales.csv', index_col=0)
alpha = 0.05
beta = 0.9

In [3]:
z_a = scipy.stats.norm.ppf(1 - alpha)
z_b = scipy.stats.norm.ppf(1 - beta)
sample_var = np.var(hist_df['Расходы'], ddof = 1)
up = 2 * sample_var * (z_a - z_b)**2
mde = 0.05 * hist_df['Расходы'].mean()
n = up /mde**2
print(n)

22530.978726988942


Размер выборки = 22531 чеоловек

### 4. Принятие решения

In [4]:
df_control = pd.read_csv('Контроль.csv', index_col=0)
df_test = pd.read_csv('Тест.csv', index_col=0)

Так как мы хотим сравнить среднее VS среднее, то можно использовать ttest.
Он предполагает, что существует дисперсия. Проверим, и поймем, что в нашем случае это условие выполняется, но дисперсии не равны. Тогда применим  t-критерий Уэлча для односторонней альтернативы.

In [5]:
p_val = scipy.stats.ttest_ind(df_control['Расходы'], df_test['Расходы'], equal_var = 'false',  alternative='greater').pvalue
if (p_val < alpha):
  print('Принимаем альтернативную гипотезу')
else:
  print('Принимаем нулевую гипотезу')

Принимаем альтернативную гипотезу


Получается, что введение автоматической системы дозвона снизит расходы на достаточном уровне. Можно вводить)

##  <ins>Задание 2.</ins>

**Контекст**

Мы провели Cust Dev и поняли, что форма авторизации для клиента сложна, и из-за этого часть клиентов отваливается в заявочном процессе. Мы решили разработать новый процесс авторизации, и у нас есть гипотеза, что он позволит повысить утилизации наших счетов.
        
**Тест**

Мы проводим тест, в рамках которого мы разбиваем весь поток на две части рановероятно, где у одной части (контроля) старый процесс авторизации, а у второй части (теста) - новый. Целью этого теста является повышение утилизаций счёта.

**Статистические вводные**

Проверка однородности выбранного параметра
осуществяется с уровнем значимости 2%.

Уровень значимости 5%.

Мы хотели бы различать 3%-ное изменение целевой метрики с вероятностью 95%.

### 1. Необходимо выбрать метрику и привести аргументацию.

**Целевая метрика** = количество утилизаций счета

По условию целью теста является повышение утилизации счета

### 2. Альтернатива в критерии.

**Гипотеза H(0)** - количество утилизаций счета не изменится 

**Альтернатива H(1)** - количество утилизаций счета увеличится на 5% и более

### 3. Нужно выбрать параметр(-ы), влияющий(-ие) на целевую метрику. Привести аргументацию.

In [6]:
card_df = pd.read_csv('hist_credit_card.csv', index_col=0)
alpha = 0.05
beta = 0.95
card_df.corr()

,Возраст,Доход клиента,Вероятность банкротства,Флаг утилизации счёта,Расходы,PV КК,PV услуги,NPV
Возраст,1.000000,0.001514,-0.004102,0.007277,0.002460,0.003447,0.006117,0.003747
Доход клиента,0.001514,1.000000,-0.000882,0.000706,0.001032,-0.002142,-0.001937,-0.002269
Вероятность банкротства,-0.004102,-0.000882,1.000000,-0.215398,-0.099635,-0.043319,-0.139128,-0.049433
Флаг утилизации счёта,0.007277,0.000706,-0.215398,1.000000,0.452036,0.204981,0.646524,0.233611
Расходы,0.002460,0.001032,-0.099635,0.452036,1.000000,0.090826,0.294538,0.086146
PV КК,0.003447,-0.002142,-0.043319,0.204981,0.090826,1.000000,0.128742,0.998232
PV услуги,0.006117,-0.001937,-0.139128,0.646524,0.294538,0.128742,1.000000,0.183529
NPV,0.003747,-0.002269,-0.049433,0.233611,0.086146,0.998232,0.183529,1.000000


Видно, что на флаг утилизации влияет параметр "Pv услуги"
 
Также близко к средней силе связи значение для параметра "Расходы" 

### 4. Каков размер выборки? 

$$n = \frac{(Z_{1 - α} - Z_{1 - β})^2 * σ^2}{k * Δ^2}$$ 

Из условия задачи знаем:
1.   Уровень α = 0.05
2.   Уровень β = 0.95
3.   k = 0.5

Не знаем σ, но можем посчитать выборочную дисперсию.

Δ = MDE

In [7]:
z_a = scipy.stats.norm.ppf(1 - alpha)
z_b = scipy.stats.norm.ppf(1 - beta)
sample_var = np.var(card_df['Флаг утилизации счёта'], ddof = 1)
up = 2 * sample_var * (z_a - z_b)**2
mde = 0.03 * card_df['Флаг утилизации счёта'].mean()
n = up /mde**2
print(n)

8382.147778769058


Размер выборки 8383 человек

### 5. Проверка на однородность, применение критерия. Принятие решения. 

Проверим однородность, по выбранному в пункте 3 параметру (PV услуги)

Однородность можно проверить с помощью критерия Андерсона-Дарлинга.

**Гипотеза H(0) для однородности** - выборки имеют одинаковое распределение

**Альтернатива H(1) для однородности** - выборки взяты из разных распределений

In [8]:
df_control = pd.read_csv('Контроль_2.csv', index_col=0)
df_test = pd.read_csv('Тест_2.csv', index_col=0)

In [9]:
p_val = scipy.stats.anderson_ksamp([df_control['PV услуги'], df_test['PV услуги']]).pvalue 
if (p_val < 0.02):
  print('Принимаем альтернативную гипотезу')
else:
  print('Принимаем нулевую гипотезу')

Принимаем нулевую гипотезу


Выборки получились однородными, поэтому применим критерий для целевой метрики.

In [10]:
stats, p_val = statsmodels.stats.proportion.proportions_ztest(
    [df_control['Флаг утилизации счёта'].sum(), df_test['Флаг утилизации счёта'].sum()], 
    [8383, 8383], 
    value=None, 
    alternative='smaller', 
    prop_var=False)
if (p_val < 0.05):
  print('Принимаем альтернативную гипотезу')
else:
  print('Принимаем нулевую гипотезу')


Принимаем нулевую гипотезу


Принимаем нулевую гипотезу, значит считаем, что количество утилизаций счета не увеличится при введении нового процесса авторизации. 